In [12]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn import datasets
import pandas as pd
from keras.models import load_model
from keras.callbacks import EarlyStopping

In [2]:
# Load Titanic dataset
df = pd.read_csv('train.csv')
df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
df.Embarked.replace('C', 0, inplace=True)
df.Embarked.replace('S', 1, inplace=True)
df.Embarked.replace('Q', 2, inplace=True)
df.Sex.replace('male', 1, inplace=True)
df.Sex.replace('female', 0, inplace=True)
df.dropna(inplace=True)


In [3]:
# Save with changes
df.to_csv('train_v2.csv', index=False)

In [4]:
# Load with changes
df = pd.read_csv('train_v2.csv')

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1.0
1,1,1,0,38.0,1,0,71.2833,0.0
2,1,3,0,26.0,0,0,7.9250,1.0
3,1,1,0,35.0,1,0,53.1000,1.0
4,0,3,1,35.0,0,0,8.0500,1.0


In [9]:
# Predictors
predictors = df.drop('Survived', axis=1).as_matrix()

# Input Shape
n_cols = predictors.shape[1]

# Convert the target to categorical: target
target = to_categorical(df.Survived)

In [10]:
# Define function to get new model
def get_new_model(n_cols=n_cols):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(n_cols, )))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    return model

In [11]:
# Import the SGD optimizer
from keras.optimizers import SGD

# Create list of learning rates: lr_to_test
lr_to_test = [0.000001, 0.01, 1]

# Loop over learning rates
for lr in lr_to_test:
    print('\n\nTesting model with learning rate: %f\n'%lr )
    
    # Build new model to test, unaffected by previous models
    model = get_new_model()
    
    # Create SGD optimizer with specified learning rate: my_optimizer
    my_optimizer = SGD(lr=lr)
    
    # Compile the model
    model.compile(optimizer=my_optimizer, loss='categorical_crossentropy')
    
    # Fit the model
    model.fit(predictors, target)
    



Testing model with learning rate: 0.000001

Epoch 1/10
712/712 [==============================] - 2s - loss: 4.1244      
Epoch 2/10
712/712 [==============================] - 0s - loss: 4.1162     
Epoch 3/10
712/712 [==============================] - 0s - loss: 4.1078     
Epoch 4/10
712/712 [==============================] - 0s - loss: 4.0995     
Epoch 5/10
712/712 [==============================] - ETA: 0s - loss: 3.9924

In [14]:
# Model Validation
model = get_new_model()

# Early Stopping Monitor
early_stopping_monitor = EarlyStopping(patience=2)

# Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(predictors, target, validation_split=0.3, epochs=20, callbacks=[early_stopping_monitor])

Train on 498 samples, validate on 214 samples
Epoch 1/20
498/498 [==============================] - 0s - loss: 2.0654 - acc: 0.5984 - val_loss: 0.6647 - val_acc: 0.7150
Epoch 2/20
498/498 [==============================] - 0s - loss: 0.7549 - acc: 0.6305 - val_loss: 0.5546 - val_acc: 0.7103
Epoch 3/20
498/498 [==============================] - 0s - loss: 0.8304 - acc: 0.6185 - val_loss: 0.5526 - val_acc: 0.7150
Epoch 4/20
498/498 [==============================] - 0s - loss: 0.6411 - acc: 0.6687 - val_loss: 0.7507 - val_acc: 0.3925

In [15]:
# Summary of the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 100)               800       
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 202       
Total params: 11,102
Trainable params: 11,102
Non-trainable params: 0
_________________________________________________________________
